# Специальные проверки

Часто наши данные довольно сложно анализировать из-за того, что в них содержатся дубликаты или незаполненные значения. Дублирующие и незаполненные поля могут привести к нежелательным дублям при объединении источников или к занулением всех значений при агрегации или математических операциях (в Python пустое значение NaN в операциях ведет себя следующим образом: `NaN + 2 == NaN`). Как найти и избавиться от таких значений?

In [1]:
import pandas as pd
orders = pd.read_excel('Superstore.xls')

## Дубликаты
Для работы с дубликатами есть 2 важные функции: `duplicated()` и `drop_duplicates()` (обе функции запускаются от имени таблицы). Функция `duplicated()` является фильтром и позволяет найти дублирующиеся строки:

In [7]:
filtr = orders[['Order ID','Customer ID']].duplicated()
orders[filtr];

In [8]:
# Найдем все строки, в которых Order ID и Customer ID повторяются
orders.loc[orders.loc[:, ['Order ID', 'Customer ID']].duplicated(), :];

**Обратите внимание!** По умолчанию функция помечает как дубликат все строки, кроме первой.

Попробуйте найти дубликаты только по столбцам 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name'. Сравните получившуюся таблицу с полученной в предыдущей ячейке. 

In [35]:
filtr = orders[['Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name']].duplicated()
orders[filtr];

Чтобы найти наоборот, не дубликаты, а не дублирующиеся значения, нужно инвертировать результат использования `duplicated()`. Для инверсии используется символ `~`.

Добавление параметра `keep=False` в функцию `duplicated(...)` позволит пометить как дубликаты все дублирующие строки, включая первую.

In [36]:
filtr = orders[['Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name']].duplicated()
orders[~filtr];

In [ ]:
# Найдем всех покупателей, которые заказали только один товар, и информацию о их покупках
orders.loc[~orders.loc[:, 'Customer Name'].duplicated(keep=False), :]

Иногда нужно просто взять таблицу и удалить из нее все дубликаты. Это делается функцией `drop_duplicates()`.

In [11]:
orders.drop_duplicates().shape

(9994, 21)

In [ ]:
orders.loc[:, ['Customer ID','Customer Name']].drop_duplicates()

Применение параметра `keep=False` также приведет к удалению всех дублирующих строк, включая первую.

На листе Returns находятся данные о возвратах, однако данные были внесены с ошибками и информация о возврате дублирована по количеству товаров в заказе. Посчитайте реальное количество возвратных заказов.

In [38]:
orders.iloc[:,1].str.isnumeric()

0       False
1       False
2       False
3       False
4       False
        ...  
9989    False
9990    False
9991    False
9992    False
9993    False
Name: Order ID, Length: 9994, dtype: bool

## Пустые ячейки

Чтобы найти незаполненные ячейки, можно использовать функцию `isna()`. Результатом выполнения этой функции будет таблица с теми же столбцами и строками, но заполненная `True` там, где значения отсутствует, и False в остальных случаях:

In [22]:
orders.isna().sum()

Row ID            0
Order ID          0
Order Date        0
Ship Date         0
Ship Mode         0
Customer ID       0
Customer Name     0
Segment           0
Country           0
City              0
State             0
Postal Code      11
Region            0
Product ID        0
Category          0
Sub-Category      0
Product Name      0
Sales             0
Quantity          0
Discount          0
Profit            0
dtype: int64

Совместное использование с функциями `all()` и `any()` позволит определить, в каких столбцах все или хотя бы одно значение отсутствует соответственно:

In [28]:
orders.notna().sum()

Row ID           9994
Order ID         9994
Order Date       9994
Ship Date        9994
Ship Mode        9994
Customer ID      9994
Customer Name    9994
Segment          9994
Country          9994
City             9994
State            9994
Postal Code      9983
Region           9994
Product ID       9994
Category         9994
Sub-Category     9994
Product Name     9994
Sales            9994
Quantity         9994
Discount         9994
Profit           9994
dtype: int64

In [23]:
orders.isna().any()

Row ID           False
Order ID         False
Order Date       False
Ship Date        False
Ship Mode        False
Customer ID      False
Customer Name    False
Segment          False
Country          False
City             False
State            False
Postal Code       True
Region           False
Product ID       False
Category         False
Sub-Category     False
Product Name     False
Sales            False
Quantity         False
Discount         False
Profit           False
dtype: bool

Это позволит нам быстро находить записи, в которых значения незаполненны. Для этого функцию `isna()` применим к интересующей нас колонке и используем как фильтр:

In [44]:
is_na = orders['Postal Code'].isna()
orders[is_na].fillna(100);

In [43]:
orders.loc[orders.loc[:, 'Postal Code'].isna(), :];

Заполнить эти значения можно по старинке присваиванием с фильтром:

In [ ]:
orders.loc[:, 'Postal Code 1'] = orders.loc[orders.loc[:, 'Postal Code'].isna(), 'Postal Code'] = 'Без индекса'
orders

Либо с помощью короткого синтаксиса `fillna()`, который автоматически подставит фильтры где нужно (удобно, когда в качестве параметра указывается колонка). `fillna()` можно использовать с таблицей целиком.

In [ ]:
orders.loc[:, 'Postal Code 2'] = orders.loc[:, 'Postal Code'].fillna('Без индекса')
orders

Если мы хотим избавиться от строк с дубликатами совсем, то можем использовать метод `dropna()`:

In [ ]:
orders = orders.dropna(subset=['Postal Code'])
orders